Imports

Análise de Rede de Julgamentos Históricos

Este notebook realiza uma análise completa sobre uma base de dados de julgamentos históricos. O objetivo é carregar os dados, pré-processá-los, construir um grafo de rede para representar as conexões entre locais e períodos, e extrair insights através de análises estatísticas e visuais.#

Usaremos a biblioteca `networkx` para modelar e analisar a rede, `pandas` para a manipulação de dados e `matplotlib` para as visualizações. Todo o código principal foi modularizado nos scripts da pasta `core`.#


## 1. Configuração e importação

In [14]:
import sys
import os
from pathlib import Path
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Markdown

# --- Configuração de Caminhos ---
current_dir = Path.cwd()
while not (current_dir / "src").exists():
    current_dir = current_dir.parent
    if current_dir == Path("/"):
        raise FileNotFoundError("Pasta 'src' não encontrada!")

src_path = current_dir / "src"
sys.path.append(str(src_path))  # Permite importar módulos do core

# --- Imports Personalizados ---
from core.data_loader import carregar_dados
from core.preprocessing import preprocessar
from core.graph_builder import construir_grafo
from core.analysis import (
    executar_analise_completa,
    calcular_taxas_mortalidade_geral,
    mostrar_taxa_mortalidade
)
from core.graph_analysis import (
    analisar_grafo,
    mostrar_nos_com_maior_grau,
    analisar_centralidades,
    densidade_rede,
    detectar_comunidades
)
from core.visualization import (
    top_paises,
    top_localizacoes,
    julgamentos_por_decada,
    distribuir_geograficamente
)

print("✅ Módulos importados com sucesso!")
print(f"📂 Caminho dos dados: {src_path/'dados.csv'}")

✅ Módulos importados com sucesso!
📂 Caminho dos dados: c:\Users\Aline\OneDrive\Documentos\GitHub\grafoWitchTrials\src\dados.csv


## 2. Carregamento e Pré-processamento dos Dados


In [17]:
# Carrega o arquivo CSV
try:
    df_raw = carregar_dados(str(src_path / "trials.csv"))
    print(f"📊 Dados carregados: {len(df_raw)} registros")
    display(df_raw.head(3))
except Exception as e:
    print(f"❌ Erro ao carregar CSV: {e}")

# Pré-processamento
df = preprocessar(df_raw)
print("\n🔍 Dados após pré-processamento:")
print(f"- Registros válidos: {len(df)}")
print(f"- Colunas: {list(df.columns)}")
display(df.head(3))

📊 Dados carregados: 10940 registros


,year,decade,century,tried,deaths,city,gadm_adm2,gadm_adm1,gadm_adm0,lon,lat,record_source
0,NaN,1520,1500,1,1.0,NaN,NaN,NaN,Estonia,NaN,NaN,Madar (1990)
1,NaN,1530,1500,1,1.0,NaN,NaN,NaN,Estonia,NaN,NaN,Madar (1990)
2,NaN,1540,1500,5,5.0,NaN,NaN,NaN,Estonia,NaN,NaN,Madar (1990)



🔍 Dados após pré-processamento:
- Registros válidos: 10940
- Colunas: ['year', 'decade', 'century', 'tried', 'deaths', 'city', 'gadm_adm2', 'gadm_adm1', 'gadm_adm0', 'lon', 'lat', 'record_source', 'is_desconhecido', 'localizacao']


,year,decade,century,tried,deaths,city,gadm_adm2,gadm_adm1,gadm_adm0,lon,lat,record_source,is_desconhecido,localizacao
0,NaN,1520,1500,1,1,NaN,NaN,NaN,Estonia,NaN,NaN,Madar (1990),True,NaN
1,NaN,1530,1500,1,1,NaN,NaN,NaN,Estonia,NaN,NaN,Madar (1990),True,NaN
2,NaN,1540,1500,5,5,NaN,NaN,NaN,Estonia,NaN,NaN,Madar (1990),True,NaN


## 3.Construção do Grafo de Relações

In [18]:
G = construir_grafo(df)

# Visualização rápida do grafo
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    with_labels=False,
    node_size=20,
    alpha=0.6,
    node_color="skyblue",
    edge_color="gray"
)
plt.title("Relações entre Localizações e Décadas", pad=20)
plt.show()


Grafo construído com:
- 968 locais
- 53 décadas
- 3588 conexões


## 4.Análise Exploratória

# 4.1. Taxas de Mortalidade

In [19]:
total_julgamentos, total_mortes, taxa = calcular_taxas_mortalidade_geral(df)
display(Markdown(f"""
**📌 Taxa de Mortalidade Geral**  
- **Total de Julgamentos:** `{total_julgamentos}`  
- **Total de Mortes:** `{total_mortes}`  
- **Taxa de Mortalidade:** `{taxa:.2%}`  
"""))


Taxa de Mortalidade Geral:
Total de Julgamentos: 43240
Total de Mortes: 16333
Taxa de Mortalidade Geral (Mortes/Julgamentos): 0.3777



**📌 Taxa de Mortalidade Geral**  
- **Total de Julgamentos:** `43240`  
- **Total de Mortes:** `16333`  
- **Taxa de Mortalidade:** `37.77%`  


# 4.2. Top Países e Localizações

In [20]:
top_paises(df, G, n=10)
top_localizacoes(df, G, n=10)

,Localização,Julgamentos,Mortes,Taxa Mortalidade
0,NaN,4048,1493,0.368824
777,"nan, Mecklenburg-Vorpommern, Germany",3844,0,0.000000
351,"nan, Scotland, United Kingdom",3563,190,0.053326
23,"nan, Vaud, Switzerland",3373,3371,0.999407
8,"nan, Fribourg, Switzerland",1230,330,0.268293
442,"nan, England, United Kingdom",1117,364,0.325873
967,"nan, Sachsen, Germany",894,300,0.335570
22,"nan, Valais, Switzerland",819,304,0.371184
12,"nan, Lucerne, Switzerland",770,167,0.216883
335,"nan, Rheinland-Pfalz, Germany",718,638,0.888579


# 4.3. Padrões Temporais por Década

In [21]:
julgamentos_por_decada(df, G)

,Julgamentos,Mortes,Taxa Mortalidade
Década,,,
1300,48,38,0.791667
1310,116,102,0.879310
1320,71,5,0.070423
1330,53,23,0.433962
1340,9,0,0.000000
1350,15,0,0.000000
1360,12,1,0.083333
1370,30,5,0.166667
1380,31,2,0.064516


# 4.4. Distribuição Geográfica

In [22]:
distribuir_geograficamente(df, G)

,Local,Julgamentos,Mortes,Latitude,Longitude,País
0,"Isny im Allgau, Baden-Wurttemberg, Germany",6,3,47.695000,10.042000,Germany
1,"Erolzheim, Baden-Wurttemberg, Germany",6,6,48.091700,10.070800,Germany
2,"Heidelberg, Baden-Wurttemberg, Germany",6,3,49.398800,8.672430,Germany
3,"Augsburg, Bayern, Germany",172,26,48.370545,10.897790,Germany
4,"Regensburg, Bayern, Germany",100,49,49.013430,12.101624,Germany
...,...,...,...,...,...,...
757,"Ortenberg, Hessen, Germany",10,0,50.355600,9.049020,Germany
758,"Langgons, Hessen, Germany",6,0,50.500000,8.666670,Germany
759,"Hitzkirchen, Hessen, Germany",1,0,50.342900,9.253200,Germany
760,"Kefenrod, Hessen, Germany",1,0,50.344700,9.210830,Germany


## 5. Análise de Redes

# 5.1. Métricas Básicas do Grafo

In [23]:
analisar_grafo(G)
mostrar_nos_com_maior_grau(G, top_n=10)
analisar_centralidades(G)
densidade_rede(G)


=== ANÁLISE ESTRUTURAL DA REDE ===
Número de nós: 1021
Número de arestas: 3588
Grau médio: 7.03
Componentes fracos conectados: 3
Tamanho do maior componente: 1017
Nó mais central (grau): 1610 - Centralidade: 0.2931

Top 10 nós com maior grau:
1610: grau 299
1620: grau 280
1590: grau 277
1600: grau 258
1580: grau 182
1630: grau 177
1660: grau 167
1650: grau 163
1570: grau 139
1670: grau 137

Top 5 nós por Centralidade de Grau:
1610: 0.2931
1620: 0.2745
1590: 0.2716
1600: 0.2529
1580: 0.1784

Top 5 nós por Centralidade de Proximidade:
1610: 0.2931
1620: 0.2745
1590: 0.2716
1600: 0.2529
1580: 0.1784

Top 5 nós por Centralidade de Intermediação:
nan: 0.0000
1520: 0.0000
1530: 0.0000
1540: 0.0000
1580: 0.0000

Densidade da rede: 0.0034


# 5.2. Detecção de Comunidades

In [24]:
detectar_comunidades(G)


=== Detecção de Comunidades ===
Total de comunidades detectadas: 968


c:\Users\Aline\OneDrive\Documentos\GitHub\grafoWitchTrials\src\core\graph_analysis.py:104: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Não está funcionando
 
# Salva o grafo para análise externa (Gephi, Cytoscape, etc.)
nx.write_gexf(G, str(src_path / "resultados" / "grafo_julgamentos.gexf"))

# Salva os dados processados
df.to_csv(str(src_path / "resultados" / "dados_processados.csv"), index=False)

print("✅ Resultados exportados em:")
print(f"- 📊 Grafo: {src_path/'resultados/grafo_julgamentos.gexf'}")
print(f"- 📂 Dados: {src_path/'resultados/dados_processados.csv'}")

## 7. Conclusã0

display(Markdown("""
## 📌 Principais Insights  

1. **Padrão Geográfico**:  
   - Países como `X` e `Y` concentraram mais julgamentos.  
   - A taxa de mortalidade foi maior em `Região Z`.  

2. **Variação Temporal**:  
   - Décadas como `AAAA` tiveram picos de atividade.  

3. **Rede de Relações**:  
   - `Local A` é o nó mais central na rede.  
   - Foram detectadas `N comunidades` geográficas.  

🔍 **Próximos passos**: Análise de correlação entre fatores socioeconômicos e taxas de julgamento.  
"""))